In [1]:
from uuid import uuid4
import os
from dotenv import load_dotenv
import pprint
import json
from datetime import datetime, timedelta, timezone

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import Qdrant
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StdOutCallbackHandler

from langchain_core.load import dumpd, dumps, load, loads


#Youtube API
from googleapiclient.discovery import build

# Youtube transcript
from langchain.document_loaders import YoutubeLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain


# Vectorial DB
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct


# CrewAI
from crewai import Agent, Task, Crew
from crewai_tools import BaseTool

# Email
import smtplib

load_dotenv()

True

In [22]:
pip list

Package                                  Version
---------------------------------------- ---------------
absl-py                                  2.0.0
aiofiles                                 22.1.0
aiohttp                                  3.9.5
aiosignal                                1.3.1
aiosqlite                                0.18.0
alembic                                  1.13.1
anaconda-anon-usage                      0.4.2
anaconda-catalogs                        0.2.0
anaconda-client                          1.12.1
anaconda-cloud-auth                      0.1.4
anaconda-navigator                       2.5.0
anaconda-project                         0.11.1
annotated-types                          0.6.0
anyio                                    4.3.0
appdirs                                  1.4.4
archspec                                 0.2.1
argon2-cffi                              21.3.0
argon2-cffi-bindings                     21.2.0
asgiref                                  

## Cargo claves

In [2]:
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')
QDRANT_URL = os.getenv('QDRANT_URL')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
YOUTUBE_API_KEY = os.getenv('YOUTUBE_API_KEY')
EMAIL_PASS = os.getenv('EMAIL_PASS')

# Transcripcion videos Youtube

In [3]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=W7K3o8HM3hs&ab_channel=JuanRam%C3%B3nRallo",
                                        add_video_info=True,
                                        language=["es"]
                                       )
transcript = loader.load()

In [4]:
transcript[0]

Document(page_content='como el sistema Público de pensiones está en crisis el gobierno español pretende tapar algunos agujeros apropiándose del ahorro jubilatorio de los futuros abogados de España veámoslo Existen esencialmente dos sistemas a través de los cuales podemos prepararnos para la jubilación el primero el sistema de reparto y el segundo el sistema de capitalización el sistema de reparto se basa en el principio redistributivo normalmente de carácter coactivo te pago yo a ti hoy a cambio de que él me pague a mí mañana y en cambio el sistema de capitalización se fundamenta en la libertad y en la responsabilidad de cada individuo de ahorrar a lo largo de su vida profesional para constituir un patrimonio del que poder vivir Cuando se jubile en España el sistema que por desgracia rige para la mayoría de españoles es el sistema Público de reparto sin embargo existen ciertos colectivos profesionales que todavía hoy disfrutan de la libertad de poder acogerse a un sistema privado de ca

## Resumen
[Documentación summarization]("https://python.langchain.com/v0.2/docs/tutorials/summarization/#stuff")

Dado que las transcripciones serán textos largos necesitaremos dividir el texto en trozos. Veremos que técnica los da mejores resultados.

- Primero dividimos la transcripción en trozos(probando diferentes tamaños):

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
texts = text_splitter.split_documents(transcript)

In [15]:
texts[1].page_content

'de la mutualidad de la abogacía y que debido a no haber efectuado aportaciones ientes a su plan personal privado de pensiones ahora mismo se encuentran con que van a cobrar una pensión que consideran demasiado Baja pues lo que están solicitando es un rescate público lo que están solicitando es poder dar el Salto ahora al sistema Público de pensiones con condiciones de jubilación mejoradas con respecto a las que tendrían si cobraran la pensión derivada del ahorro capitalizado y acumulado en sus cuentas privadas de capitalización Pero el punto verdaderamente criticable ni siquiera es que estos 2000 2500 mutualistas estén reclamando en estos momentos un rescate público tiene sentido que se articule una pasarela del sistema privado al sistema público siempre que no sea en perjuicio patrimonial del sistema público Es decir de todos aquellos que han cotizado forzosamente al sistema Público de reparto de pensiones no lo verdaderamente criticable no es esto lo verdaderamente criticable es que

In [7]:
len(texts)

2

- Definimos el LLM:

In [53]:
llm = ChatGroq(model="llama3-70b-8192",
               groq_api_key= GROQ_API_KEY)

# Map-reduce

- Creamos una cadena para hacer un resumen de cada trozo de transcripción

In [42]:
map_template = """The following is a set of texts in spanish from a video transcription
{texts}
Based on this list of texts, please identify the main themes 
Helpful Answer in Spanish:"""
map_prompt = PromptTemplate.from_template(map_template)

map_chain = LLMChain(llm=llm,
                     prompt=map_prompt)

In [43]:
map_chain

LLMChain(prompt=PromptTemplate(input_variables=['texts'], template='The following is a set of texts in spanish from a video transcription\n{texts}\nBased on this list of texts, please identify the main themes \nHelpful Answer in Spanish:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022E08F16D10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022E08D32A50>, model_name='llama3-70b-8192', groq_api_key=SecretStr('**********')))

- Creamos una cadena para hacer un resumen final de los resumenes de cada trozo

In [50]:
# Reduce
reduce_template = """The following is set of summaries in spanish:
{texts}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer in Spanish:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Run chain
reduce_chain = LLMChain(llm=llm,
                        prompt=reduce_prompt)
# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(llm_chain=reduce_chain,
                                              document_variable_name="texts")

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=10000,
)

In [53]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="texts",
    # Return the results of the map steps in the output
    return_intermediate_steps=True,
)

In [54]:
result = map_reduce_chain.invoke(texts)

Token indices sequence length is longer than the specified maximum sequence length for this model (2787 > 1024). Running this sequence through the model will result in indexing errors


In [55]:
print(result["output_text"])

Basándome en los textos, puedo identificar los siguientes temas principales consolidados:

1. **Crítica al Partido Popular y su programa electoral**: La crítica al Partido Popular y su programa electoral es un tema recurrente, destacando la falta de transparencia, sinceridad y medidas concretas.

2. **Desilusión con la política y los partidos políticos**: La desilusión con la política y los partidos políticos en general, y específicamente con el Partido Popular, es un tema destacado en varios textos.

3. **Europa y valores europeos**: La importancia de Europa, los valores europeos como la libertad, la igualdad y la democracia, y la necesidad de reafirmar la identidad europea son temas importantes.

4. **Desafíos actuales y futuro de Europa**: Los desafíos actuales que enfrenta Europa, como la demografía, la economía, la seguridad y la globalización, y la necesidad de abordarlos de manera efectiva para garantizar un futuro próspero.

5. **Democracia y Estado de Derecho**: La importancia

# Refine

En este caso se crea un resumen del primer trozo de la transcripción y se van añadiendo a este los siguientes trozos generando otro resumen refinado con la nueva información.

In [57]:
transcripcion = transcript_video("https://www.youtube.com/watch?v=Qs6BdBHiLwM&ab_channel=JuanRam%C3%B3nRallo")

In [58]:
transcripcion

[Document(page_content='Cuáles serían Los Auténticos niveles de deuda Pública del Estado español si incluyeramos en esta cifra el dinero que el estado a deuda en pensiones a los futuros pensionistas veámoslo se suele decir que la deuda Pública del Estado español se ubica alrededor del 110 por del pib de hecho ya algo por debajo sin embargo esta cifra únicamente cuantifica el dinero que adeuda el estado español por los bonos en sentido amplio del término letras bonos y obligaciones del estado que el estado ha emitido en los mercados y que los inversores han adquirido pero en esta cifra de endeudamiento público no aparecen otros pasivos con los que también carga el estado español por ejemplo la deuda con proveedores del Estado español no aparece reflejada en esta cifra ahora bien como en general la deuda que las administraciones públicas tienen con los proveedores no es un monto extraordinariamente elevado puede ser el un el 2 el 3% del pib Lo cierto es que esta omisión no distorsiona de

In [62]:
prompt_template = """Write a summary in SPANISH for a newspaper article of the following:
{text}
SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary in SPANISH\n"
    "We have provided an existing summary in spanish up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)

# Chain
chain = load_summarize_chain(llm=llm,
                             chain_type="refine",
                             question_prompt=prompt,
                             refine_prompt=refine_prompt,
                             return_intermediate_steps=False,
                             input_key="input_documents",
                             output_key="output_text",
                            )

In [63]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
texts = text_splitter.split_documents(transcripcion)

In [64]:
result = chain.invoke({"input_documents": texts}, return_only_outputs=True)

In [65]:
print(result["output_text"])

Here is a refined summary in Spanish:

**La deuda pública española es mucho más alta de lo que se cree**

La deuda pública española es mucho más alta de lo que se cree. Si se incluyera la cantidad de dinero que se debe a los futuros pensionistas, la cifra actual de alrededor del 110% del PIB se multiplicaría por cinco. Según Eurostat, el valor presente de los gastos futuros en pensiones comprometidos por el Estado español asciende a un alarmante 500% del PIB, lo que lo convierte en el país europeo con mayor deuda implícita en materia de pensiones.

Esta deuda implícita puede llevar a una visión distorsionada de la situación económica real del país. Aunque no se refleja en las estadísticas oficiales de deuda pública, es un pasivo importante que el Estado español tendrá que enfrentar en el futuro. Sin embargo, es importante tener en cuenta que el Estado puede recortar las pensiones en el futuro para hacer frente a esta deuda implícita.

Es probable que en las próximas décadas se produzca

# Función para transcripcion

In [3]:
def transcriptor(url):
    '''
    Función utilizada para la transcripción de videos y su resumen.

    Inputs:
        - url: str -> La url del video.

    Output:
        - resumen: str -> El resumen del video.
    '''

    # Cargo y transcribo:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True, language=["es"])
    transcript = loader.load()
    
    return transcript[0].page_content

# 2. Crew

## 2.1. Agents

- Scraper

In [4]:
def search_videos():
    '''
    Función que devuelve los videos de los últimos 5 días

    Input:
        - xxxx: str -->

    Output:
        - results: object --> JSON con la información obtenida(titulo, fecha y url)
    '''

    # Fecha de hace 5 días
    today = datetime(datetime.now().year, datetime.now().month, datetime.now().day, tzinfo=timezone.utc)
    two_days_ago = today - timedelta(days=5)
    date_format = two_days_ago.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
    
    youtube = build("youtube", "v3", developerKey= YOUTUBE_API_KEY)    
    request = youtube.search().list(part= "snippet",
                                    order= "date",
                                    type= "video",
                                    videoDuration= "medium",
                                    publishedAfter= date_format,
                                    channelId= "UCBLCvUUCiSqBCEc-TqZ9rGw",
                                    maxResults=5
                                   )
    
    response = request.execute()

    results = {"results": []}

    for video in response["items"]:

        title = video["snippet"]["title"]
        author = video["snippet"]["channelTitle"]
        date = video["snippet"]["publishTime"]
        url = video["id"]["videoId"]
        
        results["results"].append({"title": title,
                                   "author": author,
                                   "date": date,
                                   "url": f"https://www.youtube.com/watch?v={url}&ab_channel=JuanRam%C3%B3nRallo"
                                  })
    
    return results

In [71]:
response["items"][0]["snippet"]["channelTitle"]

'Juan Ramón Rallo'

In [5]:
results = search_videos()

In [6]:
results["results"]

[{'title': '¿Por qué Warren Buffett está saliendo de la bolsa?',
  'author': 'Juan Ramón Rallo',
  'date': '2024-08-08T13:26:25Z',
  'url': 'https://www.youtube.com/watch?v=WLgiN6pLp8k&ab_channel=JuanRam%C3%B3nRallo'},
 {'title': 'Javier Milei cierra el INADI',
  'author': 'Juan Ramón Rallo',
  'date': '2024-08-07T07:25:18Z',
  'url': 'https://www.youtube.com/watch?v=ZkzPYmblxYY&ab_channel=JuanRam%C3%B3nRallo'},
 {'title': '¿Se volatilizaron dos billones de dólares con el crash bursátil?',
  'author': 'Juan Ramón Rallo',
  'date': '2024-08-06T12:17:01Z',
  'url': 'https://www.youtube.com/watch?v=x-caVhZSLQA&ab_channel=JuanRam%C3%B3nRallo'},
 {'title': '¿Por qué ha colapsado la bolsa japonesa?',
  'author': 'Juan Ramón Rallo',
  'date': '2024-08-05T19:24:28Z',
  'url': 'https://www.youtube.com/watch?v=1dcc-O__hYg&ab_channel=JuanRam%C3%B3nRallo'},
 {'title': 'El doble rasero de Pedro Sánchez: cobarde con Maduro, agresivo con Milei',
  'author': 'Juan Ramón Rallo',
  'date': '2024-08-04T1

In [10]:
def info_videos(dias=3):
    '''
    Función que devuelve información de los videos de los últimos 3 días

    Input:
        - dias: int --> Número de días

    Output:
        - results: object --> JSON con la información obtenida(titulo, autor, fecha, url, resumen)
    '''

    # Fecha de hace 3 días
    today = datetime(datetime.now().year, datetime.now().month, datetime.now().day, tzinfo=timezone.utc)
    two_days_ago = today - timedelta(days=dias)
    date_format = two_days_ago.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
    
    youtube = build("youtube", "v3", developerKey= os.getenv('YOUTUBE_API_KEY'))
    request = youtube.search().list(part= "snippet",
                                    order= "date",
                                    type= "video",
                                    videoDuration= "medium",
                                    publishedAfter= date_format,
                                    channelId= "UCBLCvUUCiSqBCEc-TqZ9rGw",
                                    maxResults=5
                                   )
    
    response = request.execute()

    results = {"results": []}

    for video in response["items"]:

        title = video["snippet"]["title"]
        author = video["snippet"]["channelTitle"]
        date = video["snippet"]["publishTime"]
        id= video["id"]["videoId"]
        url = f"https://www.youtube.com/watch?v={id}&ab_channel=JuanRam%C3%B3nRallo"
        summary = transcriptor(url)
        
        results["results"].append({"title": title,
                                   "author": author,
                                   "date": date,
                                   "url": url,
                                   "summary": summary
                                  })
    
    return results

In [13]:
info_videos = info_videos(dias=1)

In [14]:
info_videos

{'results': [{'title': '¿Por qué Warren Buffett está saliendo de la bolsa?',
   'author': 'Juan Ramón Rallo',
   'date': '2024-08-08T13:26:25Z',
   'url': 'https://www.youtube.com/watch?v=WLgiN6pLp8k&ab_channel=JuanRam%C3%B3nRallo',
   'summary': 'Warren buffett uno de los mejores inversores de toda la historia ha decidido liquidar una porción significativa de su cartera bursátil para maximizar su posición en liquidez por qué lo ha hecho justamente ahora veámoslo [Música] este pasado lunes vivimos una de las caídas diarias más intensas de toda la historia de la bolsa japonesa una sangría que también se ex end dió en menor medida hacia las bolsas europeas y estadounidenses y como es bien sabido desde este pasado lunes la tranquilidad ha regresado Aparentemente a los mercados no en vano las bolsas de Estados Unidos de Europa y de Japón han recuperado en gran medida lo que perdieron en la fatídica jornada del lunes negro sin embargo Y más allá del pintoresco y cruento episodio bursátil ex

In [ ]:
### RESEARCHER ###

# Agent:
reasarcher = Agent(
    role="Physicist Researcher",
    goal='Understand all sections of the research paper and describe the important information',
    backstory="""You work investigating at the Physics department of the university.
                  A new paper has been published and you need to extract
                  the main information and provide the information to
                  your co-workers.You have a knack for dissecting complex
                  research papers""",
    verbose=True,
    llm= llm,
    tools=[tool_instance],
    allow_delegation=False
    )


# Tasks:
task_research = Task(
    description="""Conduct an in-depth analysis of the published research paper""",
    expected_output="""Your final answer must be a full analysis report of the research paper""",
    agent=reasarcher,
    output_file='researcher_analysis.md' 

)

- Resumidor

- Escritor

- Editor

In [ ]:
### WRITER ###

# Agent
writer = Agent(
    role="Physicist Writer",
    goal= 'Craft a detailed report to present to your co-workers',
    backstory="""You are working on writing a detailed summary of a research paper.
                  Use the content of the Physicist Researcher to develop
                  a comprehensive summary of the research paper""",
    verbose=True,
    llm= llm,
    allow_delegation=False,
)

# Tasks
task_writer = Task(
    description="""Extract the most important topics and key components of the report
                    developed by the Physicist Researcher.""",
    expected_output="""Your final answer must be a summary 
                        to understand report for your co-workers""",
    agent=writer,
    output_file='writer_summary.md' 

)

# CREWAI KICKOFF

Crew = Crew(agents=[reasarcher, writer],
            tasks=[task_research, task_writer],
            verbose=99)

# <u>LABORATORIO</u>

## Envio de mail

In [20]:
sender = "regueiro.u@gmail.com"
suscriptores = ["regueiro.urko@gmail.com"]

subject= "Noticias semanales"

with open('index.html', 'r') as file:
        html_template = file.read()
    
message = html_template

#email_body = f"Subject: {subject} \n\n {message}"

#server = smtplib.SMTP("smtp.gmail.com", 587)
#server.starttls()

#server.login(sender, EMAIL_PASS)

#server.sendmail(sender, suscriptores[0], email_body)

In [ ]:
SMTP.sendmail?

In [21]:
from email.message import EmailMessage
import smtplib

recipient = suscriptores[0]

email = EmailMessage()
email["From"] = sender
email["To"] = recipient
email["Subject"] = "Sent from Python!"
email.set_content(message, subtype="html")

#server = smtplib.SMTP_SSL(sender)
# Or if TLS is used:
server = smtplib.SMTP("smtp.gmail.com", 587)
server.starttls()

server.login(sender, EMAIL_PASS)

server.login(sender, EMAIL_PASS)
server.sendmail(sender, recipient, email.as_string())
server.quit()

(221,
 b'2.0.0 closing connection 5b1f17b1804b1-4256b0cee27sm192091595e9.48 - gsmtp')

## Carga y split de documentos

In [3]:
def doc_loader(pdf):
    pdf_loader = PyPDFLoader(pdf)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1024, chunk_overlap= 100)
    doc = pdf_loader.load_and_split(text_splitter)

    return doc

In [4]:
doc = doc_loader("docs/tfg.pdf")

## Almacenamiento en la VDB de Qdrant

In [5]:
# Creo una colección en la base de datos vectorial

client = QdrantClient(url=QDRANT_URL,
                      api_key=QDRANT_API_KEY
                     )


client.create_collection(collection_name="tfg",
                         vectors_config={"content": VectorParams(size=1536, distance=Distance.COSINE)}
                        )

True

In [6]:
# Almaceno documentos en la base de datos vectorial:

embedding = OpenAIEmbeddings()

def chunked_metadata(doc, client=client, collection_name="tfg"):

    chunked_metadata = []

    for item in doc:

      id = str(uuid4())
      content = item.page_content
      source = item.metadata["source"]
      page = item.metadata["page"]

      content_vector = embedding.embed_documents([content])[0]
      vector_dict = {"content": content_vector}

      payload = {"page_content": content,
                 "metadata": {"id": id,
                              "source": source,
                              "page": page,
                             }
                }


      metadata = PointStruct(id=id, vector=vector_dict, payload=payload)
      chunked_metadata.append(metadata)

    client.upsert(collection_name=collection_name,
                  wait=True,
                  points=chunked_metadata)

In [7]:
chunked_metadata(doc)

### Reviso que colecciones tenemos en el cluster0:

In [ ]:
client.get_collections()

In [42]:
attention_collection = client.get_collection("tfg")

print(f"Points: {attention_collection.points_count} ")

Points: 113 


In [8]:
# Hagamos una busqueda para ver que documentos son más similares

client.search(
    collection_name="tfg",
    query_vector=("content", embedding.embed_documents(["resumen"])[0]),
    with_payload=["page_content", "source"],
    limit=5
)

[ScoredPoint(id='0b349987-b9c8-4f9f-8795-b28ddbf8a430', version=0, score=0.7982359, payload={'page_content': 'para su archivo, consulta y empleo para usos académicos y de investigación con la\nmención específica al autor.\nEn Santiago de Compostela, a 6 de Febrero de 2023. Firmado,'}, vector=None, shard_key=None),
 ScoredPoint(id='b121feff-8762-492d-a0df-82223ed98710', version=0, score=0.7963041, payload={'page_content': '1. Introducción\nEn la actualidad se hace necesaria una transición energética hacia la electrifica-\nción debido a la urgencia de reducir las emisiones de CO2y la escasez, cada vez\nmayor, de los combustibles fósiles. En el ámbito de la automoción, la industria es-\ntá tratando de sustituir los vehículos de combustión (VCs) convencionales por una\nnueva generación de vehículos eléctricos (VEs) e híbridos (VHs) con el objetivo de\nreducir los gases de efecto invernadero.\nComo en toda maquinaria, los lubricantes juegan un papel fundamental en la\nvida útil de sus compo

## Inicializar la base de datos vectorial

In [3]:
# Initialize vector store
client = QdrantClient(url=QDRANT_URL,
                      api_key=QDRANT_API_KEY
                     )

embedding = OpenAIEmbeddings()

vectorstore = Qdrant(client=client,
                     collection_name="tfg",
                     embeddings=embedding,
                     vector_name="content")

In [49]:
# Downlad prompt template

rag_prompt = hub.pull("rlm/rag-prompt")

In [64]:
rag_prompt.messages[0].prompt.template

"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use four sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

## RAG Pipeline

In [4]:
rag_prompt = '''
You are an assistant for summary tasks. \
Use the following pieces of retrieved context to summarize. \
If you don't know the answer, just say that you don't know. \
"{context}" CONCISE SUMMARY: 
'''
prompt = PromptTemplate(template=rag_prompt,
                        input_variables=["context", "question"])

In [5]:
handler = StdOutCallbackHandler()
retriever = vectorstore.as_retriever()

llm = ChatGroq(model="llama3-70b-8192",
               groq_api_key= GROQ_API_KEY)

# Retrieval qa chain
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       callbacks=[handler],
                                       retriever=retriever,
                                       chain_type_kwargs={"prompt": prompt})

In [7]:
# Invoke qa_chain gpt-3.5-turbo

qa_chain.invoke({"query": "Cual es el resumen del estudio?"})["result"]



> Entering new RetrievalQA chain...

> Finished chain.


'Aunque no hay un resumen explícito en el texto, se puede inferir que el estudio se enfoca en la caracterización de lubricantes, específicamente en la relación entre la longitud de la cadena del alcohol y la conductividad eléctrica y la tensión superficial. El estudio encontró que la longitud de la cadena del alcohol juega un papel fundamental en la estabilidad de la mezcla y que el aumento de la conductividad eléctrica se debe a la formación de agregados moleculares.'

In [73]:
qa_chain.invoke({"query": "Que resultados se obtienen de la conductividad electrica"})["result"]



> Entering new RetrievalQA chain...

> Finished chain.


'Según el texto, los resultados obtenidos de la conductividad eléctrica son:\n\n* La conductividad eléctrica aumenta con la concentración de alcohol en la mezcla.\n* La longitud de la cadena del alcohol juega un papel fundamental en la estabilidad de la mezcla, y se obtienen mejores resultados con alcoholes de cadena larga.\n* La conductividad eléctrica es mayor en la mezcla de 1-Nonanol que en otras mezclas.\n* La medida de la conductividad eléctrica puede ser poco certera debido a la inestabilidad de los agregados que se forman en la disolución.\n\nNo se mencionan otros resultados específicos de la conductividad eléctrica, pero se habla de la importancia de la longitud de la cadena del alcohol y la concentración de alcohol en la mezcla.'

In [16]:
qa_chain.invoke("Can you analyze the research paper?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Can you analyze the research paper?',
 'result': "I'd be happy to try! However, I need more information about what you would like me to analyze. \n\nThe references provided appear to be a list of citations from a research paper, but without the actual paper, it's challenging to provide a comprehensive analysis.\n\nCould you please provide more context or clarify what specifically you would like me to analyze? For example, are you looking for an analysis of the methodology, results, conclusions, or something else? Additionally, do you have access to the full paper, and if so, can you share it with me?"}

In [7]:
qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt

PromptTemplate(input_variables=['question'], template='{question}')

In [6]:
# Create custom tool to use the vector store

class MyCustomTool(BaseTool):
    name: str = "Research Paper"
    description: str = "A tool used to read and understand a research paper for extracting information."

    def _run(self, argument: str):

        if argument == "string":
            return qa_chain.invoke("What is the research paper about?")
        else:
            return qa_chain.invoke(argument)

tool_instance = MyCustomTool()

In [7]:
### RESEARCHER ###

# Agent
reasarcher = Agent(
    role="Physicist Researcher",
    goal='Understand all sections of the research paper and describe the important information',
    backstory="""You work investigating at the Physics department of the university.
                  A new paper has been published and you need to extract
                  the main information and provide the information to
                  your co-workers.You have a knack for dissecting complex
                  research papers""",
    verbose=True,
    llm= llm,
    tools=[tool_instance],
    allow_delegation=False
    )


# Tasks
task_research = Task(
    description="""Conduct an in-depth analysis of the published research paper""",
    expected_output="""Your final answer must be a full analysis report of the research paper""",
    agent=reasarcher,
    output_file='researcher_analysis.md' 

)


### WRITER ###

# Agent
writer = Agent(
    role="Physicist Writer",
    goal= 'Craft a detailed report to present to your co-workers',
    backstory="""You are working on writing a detailed summary of a research paper.
                  Use the content of the Physicist Researcher to develop
                  a comprehensive summary of the research paper""",
    verbose=True,
    llm= llm,
    allow_delegation=False,
)

# Tasks
task_writer = Task(
    description="""Extract the most important topics and key components of the report
                    developed by the Physicist Researcher.""",
    expected_output="""Your final answer must be a summary 
                        to understand report for your co-workers""",
    agent=writer,
    output_file='writer_summary.md' 

)

# CREWAI KICKOFF

Crew = Crew(agents=[reasarcher, writer],
            tasks=[task_research, task_writer],
            verbose=99)

In [8]:
result = Crew.kickoff()

 [DEBUG]: == Working Agent: Physicist Researcher
 [INFO]: == Starting Task: Conduct an in-depth analysis of the published research paper


> Entering new CrewAgentExecutor chain...
Thought: I need to thoroughly read and understand the research paper to extract the main information.

Action: Research Paper
Action Input: {"argument": "string"}

> Entering new RetrievalQA chain...

> Finished chain.
 

{'query': 'What is the research paper about?', 'result': 'Based on the provided context, here is a concise summary:\n\nThe document appears to be a Final Degree Project report in Physics, titled "Characterization of Lubricants: Electrical Conductivity and Surface Tension". The author is Urko Regueiro Ramos, and the tutors are Alfredo José Amigo Pombo and Óscar Giner Rajala from the Department of Applied Physics. The report was submitted in February 2023. The references cited in the report include various scientific papers and books related to physics, chemistry, and thermodynamics.'}

Thoug